In [1]:
import cantera as ct
import numpy as np

In [2]:
gas=ct.Solution("gri30.cti")

In [6]:
ics=[13, 1, 3, 2, 47, 8]
ns=len(gas.species())
species=gas.species_names
multiindex=np.zeros(ns)
for i in range(0,len(ics),2):
    multiindex[int(ics[i])]=ics[i+1]
gas.TPX=1500,1*ct.one_atm,multiindex

In [7]:
rind=32
reaction=gas.reactions()[32]
reaction

<ThreeBodyReaction: H + O2 + M <=> HO2 + M>

In [8]:
gas.forward_rate_constants[rind]

3838115.667757274

In [23]:
k0=reaction.rate.pre_exponential_factor*gas.T**(reaction.rate.temperature_exponent)*np.exp(-reaction.rate.activation_energy/(ct.gas_constant*gas.T))

In [24]:
k0/gas.forward_rate_constants[rind]

1353.9464559585504

In [27]:
stoi=np.zeros(ns)
k=0
for third_body in range(ns):
    efficiency=reaction.default_efficiency
    stoi[third_body]+=1
    if species[third_body] in reaction.efficiencies.keys():
        efficiency=reaction.efficiencies[species[third_body]]
    #since the k are dimensionalized, we are missing a dimension here
    #concentrations should be moles per unit volume
    #we only know the mole fractions
    #is there some presumed volume for the gas??
    #well, PV=nRT..., so RT/P is natural volume per moles, indeed
    k+=efficiency*k0*np.product(gas.X**(stoi))/(ct.gas_constant*gas.T/gas.P)
    stoi[third_body]-=1

In [29]:
print(k/gas.forward_rate_constants[rind])

1.0000000000000009


In [32]:
print(gas())


  gri30:

       temperature            1500  K
          pressure          101325  Pa
           density        0.224638  kg/m^3
  mean mol. weight         27.6498  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy       1.295e+06        3.581e+07     J
   internal energy      8.4395e+05        2.333e+07     J
           entropy          9211.2        2.547e+05     J/K
    Gibbs function     -1.2522e+07       -3.462e+08     J
 heat capacity c_p          1453.3        4.018e+04     J/K
 heat capacity c_v          1152.6        3.187e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
                O2       0.181818         0.210416         -29.4885
               CH4      0.0909091        0.0527465         -35.8316
                N2       0.727273         0.736838         -26.3202
     [  +50 minor]        

In [20]:
ct.gas_constant*gas.T/gas.P

123.08604145077719